<a href="https://colab.research.google.com/github/Forbusinessuseyukikoishiguro/-/blob/main/%E3%80%90note%E5%85%AC%E9%96%8B%E3%80%91_%E5%B0%8F%E5%AD%A6%E7%94%9F%E5%90%91%E3%81%91%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0%E3%82%B9%E3%82%AF%E3%83%BC%E3%83%AB_AI%E3%82%AB%E3%83%AA%E3%82%AD%E3%83%A5%E3%83%A9%E3%83%A0%E4%BD%9C%E6%88%90%E3%83%84%E3%83%BC%E3%83%AB20250527.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 🤖 改良版 Google Colab対応 プログラミングスクールAI
# 軽量化・高速化・エラー修正版

# 必要なライブラリのインストール
!pip install openai pandas matplotlib seaborn plotly ipywidgets

import openai
from openai import OpenAI
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import random
import re
from typing import List, Dict, Any
from IPython.display import display, HTML, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import Button, VBox, HBox, HTML, Output, Text, Textarea
import os
import time

# Google Colab Secretsのインポート
try:
    from google.colab import userdata
    COLAB_AVAILABLE = True
    print("✅ Google Colab環境を検出しました")
except ImportError:
    COLAB_AVAILABLE = False
    print("ℹ️ Google Colab環境ではありません")

# OpenAIクライアントのグローバル変数
client = None

def quick_test():
    """診断テスト関数"""
    print("🧪 システム診断テスト")
    print("-" * 30)

    # 基本ライブラリのテスト
    try:
        import openai
        print("✅ OpenAI ライブラリ: OK")
    except ImportError:
        print("❌ OpenAI ライブラリ: NG")

    try:
        import ipywidgets
        print("✅ ipywidgets: OK")
    except ImportError:
        print("❌ ipywidgets: NG")

    # Colab環境チェック
    if COLAB_AVAILABLE:
        print("✅ Google Colab: OK")
    else:
        print("⚠️ Google Colab: 標準環境ではありません")

    print("\n✅ 診断完了 - start_gui()で開始可能")

def setup_openai_api():
    """OpenAI APIキーを安全に設定する関数"""
    global client
    print("🔐 OpenAI APIキーの設定中...")

    if COLAB_AVAILABLE:
        try:
            api_key = userdata.get('OPENAI_API_KEY')
            client = OpenAI(api_key=api_key)
            print("✅ Google Colab SecretsからAPIキーを読み込みました")
            return True
        except Exception as e:
            print(f"❌ Secretsからの読み込みに失敗: {e}")
            print("\n🔧 設定方法:")
            print("1. 左サイドバーの🔑（鍵アイコン）をクリック")
            print("2. 'Add new secret'をクリック")
            print("3. Name: OPENAI_API_KEY")
            print("4. Value: あなたのOpenAI APIキー")
            print("5. 'Notebook access'をONにする")
            return False
    else:
        api_key = os.getenv('OPENAI_API_KEY')
        if api_key:
            client = OpenAI(api_key=api_key)
            print("✅ 環境変数からAPIキーを読み込みました")
            return True
        else:
            print("❌ APIキーが見つかりません")
            print("環境変数 OPENAI_API_KEY を設定してください")
            return False

class SimpleProgrammingSchoolGUI:
    """軽量化されたプログラミングスクールAIツール"""

    def __init__(self):
        """初期化"""
        self.current_data = {}
        self.output = Output()

        # データ定義（簡略化）
        self.functions = {
            "📚 カリキュラム生成": self.generate_curriculum,
            "📖 レッスンプラン作成": self.create_lesson_plan,
            "🎯 クイズ生成": self.generate_quiz,
            "🚀 プロジェクト提案": self.create_project_ideas
        }

        self.grades = ["低学年 (6-8歳)", "中学年 (9-10歳)", "高学年 (11-12歳)"]
        self.topics = ["Scratch基礎", "ゲーム制作", "ロボット制御", "Webサイト作成", "データ活用"]

    def create_button(self, text, callback, style='info', width='300px'):
        """ボタン作成"""
        btn = Button(
            description=text,
            button_style=style,
            layout=widgets.Layout(width=width, height='35px', margin='3px')
        )
        btn.on_click(callback)
        return btn

    def show_main_menu(self):
        """メインメニュー表示"""
        with self.output:
            clear_output(wait=True)
            print("🤖 プログラミングスクール AI アシスタント")
            print("=" * 50)

        buttons = []
        for func_name, func in self.functions.items():
            btn = self.create_button(func_name, lambda b, f=func: self.select_function(f), 'primary', '400px')
            buttons.append(btn)

        exit_btn = self.create_button("終了", self.exit_app, 'danger', '200px')

        display(VBox([
            HTML("<h3>🎯 機能を選択してください</h3>"),
            VBox(buttons),
            HBox([exit_btn], layout=widgets.Layout(justify_content='center'))
        ]))

    def select_function(self, func):
        """機能選択"""
        self.current_function = func
        self.show_grade_selection()

    def show_grade_selection(self):
        """学年選択"""
        with self.output:
            clear_output(wait=True)
            print("📚 学年を選択してください")

        buttons = []
        for grade in self.grades:
            btn = self.create_button(grade, lambda b, g=grade: self.select_grade(g), 'success')
            buttons.append(btn)

        back_btn = self.create_button("戻る", lambda b: self.show_main_menu(), 'warning')

        display(VBox([
            HTML("<h3>📚 対象学年を選択</h3>"),
            VBox(buttons),
            HBox([back_btn], layout=widgets.Layout(justify_content='center'))
        ]))

    def select_grade(self, grade):
        """学年選択処理"""
        self.current_data['grade'] = grade
        self.show_topic_selection()

    def show_topic_selection(self):
        """トピック選択"""
        with self.output:
            clear_output(wait=True)
            print("📝 学習テーマを選択してください")

        buttons = []
        for topic in self.topics:
            btn = self.create_button(topic, lambda b, t=topic: self.select_topic(t), 'info')
            buttons.append(btn)

        back_btn = self.create_button("戻る", lambda b: self.show_grade_selection(), 'warning')

        display(VBox([
            HTML("<h3>📝 学習テーマを選択</h3>"),
            VBox(buttons),
            HBox([back_btn], layout=widgets.Layout(justify_content='center'))
        ]))

    def select_topic(self, topic):
        """トピック選択処理"""
        self.current_data['topic'] = topic
        self.execute_function()

    def execute_function(self):
        """機能実行"""
        with self.output:
            clear_output(wait=True)
            print("🤖 AI処理中...")
            print("少々お待ちください... ☕")

        # シンプルなプログレス表示
        progress = widgets.IntProgress(
            value=0, min=0, max=100,
            description='処理中:', bar_style='info'
        )
        display(progress)

        try:
            # プログレスバー更新
            for i in range(0, 101, 20):
                progress.value = i
                time.sleep(0.3)

            # AI処理実行
            result = self.current_function()

            # 結果表示
            with self.output:
                clear_output(wait=True)
                self.display_result(result)

        except Exception as e:
            with self.output:
                clear_output(wait=True)
                print(f"❌ エラーが発生しました: {str(e)}")
                print("APIキーの設定やネットワーク接続を確認してください")

        # メインメニューに戻るボタン
        back_btn = self.create_button("メインメニューに戻る", lambda b: self.show_main_menu(), 'primary')
        display(HBox([back_btn], layout=widgets.Layout(justify_content='center')))

    def display_result(self, result):
        """結果表示"""
        print("🎉 処理完了！")
        print("=" * 40)

        if isinstance(result, dict) and "error" in result:
            print(f"❌ エラー: {result['error']}")
            return

        if isinstance(result, str):
            print(result)
        elif isinstance(result, dict):
            for key, value in result.items():
                print(f"{key}: {value}")
        elif isinstance(result, list):
            for i, item in enumerate(result, 1):
                print(f"{i}. {item}")

    # AI機能実装（簡略化版）
    def generate_curriculum(self):
        """カリキュラム生成"""
        try:
            prompt = f"""
            小学生向けプログラミングカリキュラムを作成してください。

            対象: {self.current_data['grade']}
            テーマ: {self.current_data['topic']}

            以下の内容を含む簡潔なカリキュラムを作成：
            1. 学習目標（3つ）
            2. 週次レッスン（4週間分）
            3. 評価方法

            シンプルで分かりやすい形式で出力してください。
            """

            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=800
            )

            return response.choices[0].message.content

        except Exception as e:
            return {"error": f"カリキュラム生成エラー: {str(e)}"}

    def create_lesson_plan(self):
        """レッスンプラン作成"""
        try:
            prompt = f"""
            {self.current_data['grade']}向けの{self.current_data['topic']}のレッスンプランを作成してください。

            以下の構成で60分の授業計画を作成：
            1. 導入（10分）
            2. 展開（40分）
            3. まとめ（10分）

            各段階での活動内容と指導のポイントを含めてください。
            """

            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=600
            )

            return response.choices[0].message.content

        except Exception as e:
            return {"error": f"レッスンプラン作成エラー: {str(e)}"}

    def generate_quiz(self):
        """クイズ生成"""
        try:
            prompt = f"""
            {self.current_data['grade']}向けの{self.current_data['topic']}に関するクイズを5問作成してください。

            各問題について：
            - 問題文
            - 選択肢（4つ）
            - 正解
            - 簡単な解説

            小学生が理解しやすい言葉で作成してください。
            """

            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=800
            )

            return response.choices[0].message.content

        except Exception as e:
            return {"error": f"クイズ生成エラー: {str(e)}"}

    def create_project_ideas(self):
        """プロジェクトアイデア作成"""
        try:
            prompt = f"""
            {self.current_data['grade']}向けの{self.current_data['topic']}を使ったプロジェクトアイデアを3つ提案してください。

            各アイデアについて：
            - プロジェクト名
            - 概要
            - 必要な時間
            - 学習効果
            - 難易度

            創造性を刺激する楽しいプロジェクトを提案してください。
            """

            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.8,
                max_tokens=600
            )

            return response.choices[0].message.content

        except Exception as e:
            return {"error": f"プロジェクトアイデア作成エラー: {str(e)}"}

    def exit_app(self, b):
        """アプリ終了"""
        with self.output:
            clear_output(wait=True)
            print("👋 ご利用ありがとうございました！")
            print("また start_gui() で開始できます。")

    def start(self):
        """GUI開始"""
        display(self.output)
        self.show_main_menu()

# メイン関数
def start_gui():
    """改良版GUIを開始"""
    print("🚀 改良版プログラミングスクールAIを起動中...")
    print("=" * 50)

    # APIキー設定
    if setup_openai_api():
        print("✅ APIキーの設定が完了しました")

        # 接続テスト（軽量化）
        try:
            test_response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": "Hello"}],
                max_tokens=5
            )
            print("✅ API接続確認完了")
            print("\n🎉 準備完了！ボタンをクリックして操作してください。")
            print("=" * 50)

            # GUI開始
            gui = SimpleProgrammingSchoolGUI()
            gui.start()

        except Exception as e:
            print(f"❌ API接続エラー: {e}")
            print("APIキーまたはネットワーク接続を確認してください")
    else:
        print("❌ APIキー設定に失敗しました")

# 使用方法
print("🤖 改良版 プログラミングスクールAI")
print("=" * 40)
print("✨ 改良点:")
print("- 処理速度の高速化")
print("- エラーハンドリングの強化")
print("- UIの簡略化")
print("- メモリ使用量の削減")
print()
print("🚀 使用方法:")
print("1. quick_test()  # システム診断")
print("2. start_gui()   # GUI開始")
print("=" * 40)

✅ Google Colab環境を検出しました
🤖 改良版 プログラミングスクールAI
✨ 改良点:
- 処理速度の高速化
- エラーハンドリングの強化
- UIの簡略化
- メモリ使用量の削減

🚀 使用方法:
1. quick_test()  # システム診断
2. start_gui()   # GUI開始


In [6]:
#まず診断テストを実行※コメントアウトして実行してください
#quick_test()


#実行
start_improved_gui()

NameError: name 'start_improved_gui' is not defined

In [4]:
#まず診断テストを実行※コメントアウトして実行してください
# quick_test() # This function is not defined

#実行
start_gui() # Changed from start_improved_gui()

🚀 改良版プログラミングスクールAIを起動中...
🔐 OpenAI APIキーの設定中...
✅ Google Colab SecretsからAPIキーを読み込みました
✅ APIキーの設定が完了しました
✅ API接続確認完了

🎉 準備完了！ボタンをクリックして操作してください。


Output()

IntProgress(value=0, bar_style='info', description='処理中:')

IntProgress(value=0, bar_style='info', description='処理中:')